In [1]:
import os
import spotipy
import music_tag
from spotipy.oauth2 import SpotifyClientCredentials
from syrics.api import Spotify

In [2]:
cid = 'ad89004e3ede4c50a73c6a8dd4d33718'
secret = 'aa09d86eaf644fd2a4d997ca0dac7cfc'

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

sp_dc_key = 'AQCbN61HvOcVYIy91ATyuhycbR1f3ByrHnavCRkFFXTtrP_uw5Pql_fsoyPn4p6XTSpsM8vg-FNKKOXqiPNXGgWGSCTzdsHHtwaX-9c2ljlKM_A2I-02wFPCepW486Ygknwcvq2NqcVJvaZbIOueYPl612k-NG0'
sp_lyric = Spotify(sp_dc_key)

In [3]:
all_songs = os.listdir('All songs')
song_dir = 'All songs/'
image_dir = 'All images/'

In [5]:
for song in all_songs:
    audio_file = music_tag.load_file(song_dir + song)
    title_and_artist = song.split(' - ')
    title = title_and_artist[0]
    artist_split = ' - '.join(title_and_artist[1:]) if len(title_and_artist) > 2 else title_and_artist[1]
    artist = ".".join(artist_split.split('.')[:-1])
    image_file = image_dir + title + ' - ' + artist + ".jpg"
    audio_file['tracktitle'] = title
    audio_file['artist'] = artist
    with open(image_file, 'rb') as img:
        audio_file['artwork'] = img.read()

    search_result = sp.search('https://api.spotify.com/v1/search?q=track:"' + title + '"%20artist:"' + artist + '"&type=track')
    if len(search_result['tracks']['items']) == 0:
        print(song, "fetching failed")
        continue
    song_id = search_result['tracks']['items'][0]['id']
    song_search = sp.track(song_id)
    
    track_number = song_search['track_number']
    album = song_search['album']['name']
    album_artists = ', '.join([artist['name'] for artist in song_search['album']['artists']])
    year = song_search['album']['release_date'][:4]
    
    lyrics = ""
    lyrics_response = sp_lyric.get_lyrics(song_id)
    if lyrics_response:
        for line in lyrics_response['lyrics']['lines']:
            diff = int(line['endTimeMs']) - int(line['startTimeMs'])
            lyrics = lyrics + line['words'] + "\n"
            if lyrics != "" and diff > 7000:
                lyrics += "\n"

    audio_file['year'] = year
    audio_file['album'] = album
    audio_file['tracknumber'] = track_number
    audio_file['albumartist'] = album_artists
    audio_file['lyrics'] = lyrics

    audio_file.save()


# Testing

In [25]:
model_song = music_tag.load_file('/home/neeraj/Music/Space Song.m4a')

In [26]:
model_song.tag_map

{'tracktitle': TAG_MAP_ENTRY(getter='©nam', setter='©nam', remover=None, type=<class 'str'>, sanitizer=None),
 'artist': TAG_MAP_ENTRY(getter='©ART', setter='©ART', remover=None, type=<class 'str'>, sanitizer=None),
 'album': TAG_MAP_ENTRY(getter='©alb', setter='©alb', remover=None, type=<class 'str'>, sanitizer=None),
 'albumartist': TAG_MAP_ENTRY(getter='aART', setter='aART', remover=None, type=<class 'str'>, sanitizer=None),
 'composer': TAG_MAP_ENTRY(getter='©wrt', setter='©wrt', remover=None, type=<class 'str'>, sanitizer=None),
 'tracknumber': TAG_MAP_ENTRY(getter=<function get_tracknum at 0x7ffbd6cc7520>, setter=<function set_tracknum at 0x7ffbd6cc75b0>, remover=None, type=<class 'int'>, sanitizer=None),
 'totaltracks': TAG_MAP_ENTRY(getter=<function get_totaltracks at 0x7ffbd6cc7640>, setter=<function set_totaltracks at 0x7ffbd6cc76d0>, remover=None, type=<class 'int'>, sanitizer=None),
 'discnumber': TAG_MAP_ENTRY(getter=<function get_discnum at 0x7ffbd6cc7760>, setter=<functi

# Manual Filling

In [63]:
theid = '7ucAyhKPjKsLrP14q4mcyo'
song_search = sp.track(theid)

In [64]:
track_number = song_search['track_number']
album = song_search['album']['name']
album_artists = ', '.join([artist['name'] for artist in song_search['album']['artists']])
year = song_search['album']['release_date'][:4]

In [65]:
track_number, album, album_artists, year

(1, 'Blank', 'Disfigure', '2013')

In [66]:
song = 'Blank - Disfigure.m4a'
audio_file = music_tag.load_file('All songs/Blank - Disfigure.m4a')
title_and_artist = song.split(' - ')
title = title_and_artist[0]
artist_split = ' - '.join(title_and_artist[1:]) if len(title_and_artist) > 2 else title_and_artist[1]
artist = ".".join(artist_split.split('.')[:-1])
image_file = "All images/" + title + ' - ' + artist + ".jpg"
audio_file['tracktitle'] = title
audio_file['artist'] = artist
with open(image_file, 'rb') as img:
    audio_file['artwork'] = img.read()

song_search = sp.track(theid)

track_number = song_search['track_number']
album = song_search['album']['name']
album_artists = ', '.join([artist['name'] for artist in song_search['album']['artists']])
year = song_search['album']['release_date'][:4]

lyrics = ""
lyrics_response = sp_lyric.get_lyrics(song_id)
if lyrics_response:
    for line in lyrics_response['lyrics']['lines']:
        diff = int(line['endTimeMs']) - int(line['startTimeMs'])
        lyrics = lyrics + line['words'] + "\n"
        if lyrics != "" and diff > 7000:
            lyrics += "\n"

audio_file['album'] = album
audio_file['tracknumber'] = track_number
audio_file['albumartist'] = album_artists
audio_file['year'] = year

#audio_file.save()